<a href="https://colab.research.google.com/github/exson6969/BBL2021-analysis/blob/main/espncrickinfo_scorecard_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
page = requests.get('https://www.espncricinfo.com/series/big-bash-league-2021-22-1269637/match-schedule-fixtures-and-results')

html_data = page.content

soup = BeautifulSoup(html_data)

match_page_list = []
for a in soup.find_all('a', href=True):
  if 'full-scorecard' in a['href']:
    match_page_list.append(a['href'])
print(match_page_list)
print(len(match_page_list))

['/series/big-bash-league-2021-22-1269637/sydney-sixers-vs-melbourne-stars-1st-match-1269638/full-scorecard', '/series/big-bash-league-2021-22-1269637/sydney-thunder-vs-brisbane-heat-2nd-match-1269639/full-scorecard', '/series/big-bash-league-2021-22-1269637/melbourne-renegades-vs-adelaide-strikers-3rd-match-1269640/full-scorecard', '/series/big-bash-league-2021-22-1269637/hobart-hurricanes-vs-sydney-sixers-4th-match-1269641/full-scorecard', '/series/big-bash-league-2021-22-1269637/perth-scorchers-vs-brisbane-heat-5th-match-1269642/full-scorecard', '/series/big-bash-league-2021-22-1269637/adelaide-strikers-vs-melbourne-renegades-6th-match-1269643/full-scorecard', '/series/big-bash-league-2021-22-1269637/melbourne-stars-vs-sydney-thunder-7th-match-1269644/full-scorecard', '/series/big-bash-league-2021-22-1269637/sydney-sixers-vs-hobart-hurricanes-8th-match-1269645/full-scorecard', '/series/big-bash-league-2021-22-1269637/perth-scorchers-vs-adelaide-strikers-9th-match-1269646/full-scorec

In [ ]:
emt_bat_df = pd.DataFrame()
emt_ball_df = pd.DataFrame()

for subUrl in match_page_list:
  try:
    t1_batting = pd.read_html(f"https://www.espncricinfo.com/{subUrl}")[0]
    t2_bowling = pd.read_html(f"https://www.espncricinfo.com/{subUrl}")[1]
    t2_batting = pd.read_html(f"https://www.espncricinfo.com/{subUrl}")[2]
    t1_bowling = pd.read_html(f"https://www.espncricinfo.com/{subUrl}")[3]

    merg_batting = pd.concat([t1_batting, t2_batting],ignore_index=True)
    merg_batting = merg_batting[merg_batting["BATTING"].str.contains("Did not bat") == False]
    merg_batting = merg_batting[merg_batting["BATTING"].str.contains("Fall of wickets:") == False]
    merg_batting = merg_batting[merg_batting["BATTING"].str.contains("Extras") == False]
    merg_batting = merg_batting[merg_batting["BATTING"].str.contains("TOTAL") == False]
    merg_batting = merg_batting.dropna(axis=1, how='all')
    emt_bat_df = pd.concat([emt_bat_df, merg_batting],ignore_index=True)

    merg_bowling = pd.concat([t1_bowling, t2_bowling],ignore_index=True)
    merg_bowling = merg_bowling[~merg_bowling.M.str.contains(" ", na=False)]
    emt_ball_df = pd.concat([emt_ball_df, merg_bowling],ignore_index=True)
  except:
    print(subUrl)
    

final_data = pd.concat([emt_bat_df, emt_ball_df], axis=1,ignore_index=True)
print(final_data)

/series/big-bash-league-2021-22-1269637/sydney-sixers-vs-melbourne-renegades-29th-match-1269667/full-scorecard
                       0                        1   2   3   4  5  6       7   \
0         Josh Philippe †  c Cartwright b Rainbird  83  47  75  8  3  176.59   
1             James Vince        c Zampa b Maxwell  44  29  39  6  0  151.72   
2    Moises Henriques (c)                  not out  76  38  49  8  3  200.00   
3           Dan Christian     c Cartwright b Couch   5   5   8  1  0  100.00   
4              Tom Curran         c Larkin b Couch   0   1   3  0  0    0.00   
..                    ...                      ...  ..  ..  .. .. ..     ...   
989           Sean Abbott        c Patterson b Tye   1   9  12  0  0   11.11   
990         Ben Dwarshuis          c †Inglis b Tye   0   2   6  0  0    0.00   
991          Jay Lenton †                  not out  10   9  15  1  0  111.11   
992           Nathan Lyon             b Richardson   3   3   6  0  0  100.00   
993      

In [ ]:
final_data.to_csv('bblData.csv')